In [47]:
class SOSGame:
    def __init__(self, n=8, mode="Simple"):
        self.n = n
        self.mode = mode
        self.board = [['' for _ in range(n)] for _ in range(n)]
        self.current_player = 'Blue'
        self.symbol_choice = {'Blue': 'S', 'Red': 'O'}
        self.scores = {'Blue': 0, 'Red': 0}
        self.winner = None
        self.game_over = False

    def place_symbol(self, row, col):
        if self.board[row][col] == '' and not self.game_over:
            symbol = self.symbol_choice[self.current_player]
            self.board[row][col] = (symbol, self.current_player)

            if self.mode == "Simple" and self.check_three_by_player(row, col):
                self.winner = self.current_player
                self.game_over = True
            elif self.mode == "General":
                sos_count = self.check_sos(row, col)
                self.scores[self.current_player] += sos_count
                self.check_three_and_add_score(row, col)
                if self.is_board_full():
                    self.winner = self.determine_winner()
                    self.game_over = True
            
            # Check for draw in Simple mode if board is full and no winner
            elif self.mode == "Simple" and self.is_board_full() and not self.winner:
                self.winner = "Draw"
                self.game_over = True

            if not self.game_over:
                self.switch_turn()
            elif self.winner == "Draw":
                print("It's a Draw!")

    def switch_turn(self):
        self.current_player = 'Red' if self.current_player == 'Blue' else 'Blue'

    def swap_roles(self):
        if self.symbol_choice['Blue'] == 'S':
            self.symbol_choice = {'Blue': 'O', 'Red': 'S'}
        else:
            self.symbol_choice = {'Blue': 'S', 'Red': 'O'}
        print(f"Roles swapped: Blue = {self.symbol_choice['Blue']}, Red = {self.symbol_choice['Red']}")

    def set_game_mode(self, mode):
        self.mode = mode
        print(f"Game mode set to: {mode}")

    def set_board_size(self, new_size):
        try:
            new_size = int(new_size)
            if new_size > 2:
                self.n = new_size
                self.board = [['' for _ in range(new_size)] for _ in range(new_size)]
                print(f"Board size updated to: {new_size}")
                return True
        except ValueError:
            pass
        return False

    def reset_game(self):
        self.board = [['' for _ in range(self.n)] for _ in range(self.n)]
        self.current_player = 'Blue'
        self.scores = {'Blue': 0, 'Red': 0}
        self.winner = None
        self.game_over = False
        print("Game reset")

    def check_three_by_player(self, row, col):
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        player = self.current_player
        for dr, dc in directions:
            count = 1
            for dir in [1, -1]:
                r, c = row, col
                while True:
                    r += dir * dr
                    c += dir * dc
                    if 0 <= r < self.n and 0 <= c < self.n and self.board[r][c] and self.board[r][c][1] == player:
                        count += 1
                    else:
                        break
            if count >= 3:
                return True
        return False

    def check_three_and_add_score(self, row, col):
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        player = self.current_player
        for dr, dc in directions:
            count = 1
            for dir in [1, -1]:
                r, c = row, col
                while True:
                    r += dir * dr
                    c += dir * dc
                    if 0 <= r < self.n and 0 <= c < self.n and self.board[r][c] and self.board[r][c][1] == player:
                        count += 1
                    else:
                        break
            if count >= 3:
                self.scores[player] += 1

    def check_sos(self, row, col):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                      (-1, -1), (-1, 1), (1, -1), (1, 1)]
        count = 0
        for dr, dc in directions:
            try:
                if (0 <= row+dr < self.n and 0 <= row-dr < self.n and
                    0 <= col+dc < self.n and 0 <= col-dc < self.n):
                    if (self.board[row-dr][col-dc][0] == 'S' and 
                        self.board[row][col][0] == 'O' and 
                        self.board[row+dr][col+dc][0] == 'S'):
                        count += 1
            except (IndexError, TypeError):
                continue
        return count

    def is_board_full(self):
        return all(all(cell != '' for cell in row) for row in self.board)

    def determine_winner(self):
        if self.scores['Blue'] > self.scores['Red']:
            return 'Blue'
        elif self.scores['Red'] > self.scores['Blue']:
            return 'Red'
        else:
            return 'Draw'